In [7]:
# import libraries
from bs4 import BeautifulSoup
import requests

url  = 'https://ph.hotels.com/search.do?q-destination=Philippines&destination-id=10233139&as-shown=false&q-localised-check-in=06%2F04%2F2021&q-check-in=2021-04-06&q-localised-check-out=07%2F04%2F2021&q-check-out=2021-04-07&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&intlid=modular%20country%20landing%20%3A%3A%20SQM'
page = requests.get(url)             # connect to we'bsite
#soup = BeautifulSoup(page.contents, 'html.parser')   # vanilla call
soup = BeautifulSoup(page.text)                # use instead if returning different html formats, which is in my case

In [8]:
# hotel_cards :: BeautifulSoup Object per hotel card in site, with tag li
hotel_cardsBS = soup.select_one('body div#root ._24UiGk ._3umLRC ._3xUi2p ._3N2dPv ._1myav2 .RO5S5p ul._3f26d2')   # BeautifulSoup object of containing all cards

In [9]:
hotel_cards = hotel_cardsBS.find_all('li') # creates one BS obj per card from hotel card BeautifulSoup
print('we have {0} distinct hotels'.format(len(hotel_cards)))

we have 74 distinct hotels


<h3> <b> Inspecting one of the hotel cards </b> </h3>
Upon inspection the ff were idd.
<li> an a tag with further data given by the attribute href
<li> address: in _2oHhXM contains address with format: specific, town/city, zip, province, Philippines 
<li> amenities: in ?
<li> price: in ? #rate is one day for 2 persons
<li> h ref has it all, must go to href for number of rooms
<blockquote>
    <li> number of rooms in:
    <li> customer reviews in:
    <li> add landmarks, and getting around: in 
</blockquote>

In [10]:
# try getting information from 1 hotel card
# proof of concept
hotel_sample = hotel_cards[0]

In [11]:
hotel_name = hotel_sample.find('a')['aria-label']
hotel_name


'Conrad Manila'

In [18]:
rating = hotel_sample.find(class_='_1biq31').get_text()
rating

'9.0.'

In [134]:
amneties = hotel_sample.find(class_='_1CyeN6')
amneties_col = amneties.select('li') # generates a list of all lis

get_amnety = lambda bs4_obj: bs4_obj.select('span')[-1].get_text()  # the amnety text is in the last entry with tag span

amneties_col = [get_amnety(amnety) for amnety in amneties_col]      # process amneties collection further
amneties_col

['Pool', 'Free parking', 'Airport transfer', 'Spa', 'Gym', 'Restaurant']

In [17]:
address = hotel_sample.find(class_='_2oHhXM').get_text()
address

'Seaside Boulevard, Corner of Coral Way, Pasay, 1300, Manila, Philippines'

In [39]:
price = hotel_sample.find('span', class_='_2R4dw5').get_text()
price   # preprocess mo pa to

'P6,699'

In [76]:
href = hotel_sample.find('a')['href']
href_url = 'https://ph.hotels.com'+ href

# open the page and get the ff
# number of roms
# customer review
# landmarks

page_href = requests.get(href_url)
soup_href = BeautifulSoup(page_href.content, 'html.parser')

In [85]:
# class _1O0SKi has landmarks and get around         # get BS4 sub cats of this 2
# class _1O0SKi with id info5 has hotel size

body_href = soup_href.select_one('body div#root ._24UiGk ._3umLRC ._1rVrVi')
# body_href, works

In [104]:
hotel_size = soup_href.find(id='info5', class_='_1O0SKi')   # this is a bs4 object containing which has a child containing num of rooms
# info_href, works

room_count = hotel_size.select('._1_22qw ._2Ut1Ms ul li')[0]
room_count

<li>This hotel has 347 rooms</li>

In [176]:
# landmarks 
lndmrk_trnspo = soup_href.select_one('._1O0SKi')   # has children describing landmarks and transportation available

In [178]:
landmarks_bs4 = lndmrk_trnspo.select('ul li')      # the landmarks nearest to the place
landmarks = [landmark.get_text() for landmark in landmarks_bs4]

In [185]:
# transpo is further divided into categories
# pre process transpo
transpo_categories = lndmrk_trnspo.select('._2Ut1Ms div')      # outputs all categories and its transportation subs

[<div><ul class="airport"><li>Ninoy Aquino Intl. Airport (MNL) - <span class="distance-to-location"><span class="distance-in-time">20 min drive</span><span class="distance"></span></span> </li></ul><ul class="train-station"><li>Manila Buenidia Station - <span class="distance-to-location"><span class="distance-in-time">6 min drive</span><span class="distance"></span> </span></li><li>Manila Vito Cruz Station - <span class="distance-to-location"><span class="distance-in-time">7 min drive</span><span class="distance"></span> </span></li><li>Manila Paco Station - <span class="distance-to-location"><span class="distance-in-time">7 min drive</span><span class="distance"></span> </span></li></ul><ul class="shuttle"><li>Airport shuttle (surcharge)</li></ul></div>,
 <div></div>]

In [8]:
# hotel_cards :: BeautifulSoup Object per hotel card in site, with tag li
hotel_cardsBS = soup.select_one('body div#root ._24UiGk ._3umLRC ._3xUi2p ._3N2dPv ._1myav2 .RO5S5p ul._3f26d2')   # BeautifulSoup object of containing all cards

In [ ]:
print(
'''
hotel name                  | 
address                     |
rating                      |
amneties                    |
number of rooms             |
price (2 person/ 1 day)     |
reviews                     |
landmarks                   |
'''
).format(
    hotel_name,
    rating,
    amneties,
    room_count,
    price,
    reviews,
    landmarks)